In [123]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [124]:
block_size=8
batch_size=4
max_iter=10000
learning_rate=3e-4
n_embd=384
n_layer=6
n_head=4
dropout=0.2

In [125]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text=f.read()

In [126]:
char=sorted(set(text))
print(' the length of char is {}'.format(len(char)))
print(char)
vocab_size=len(char)

 the length of char is 80
['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [127]:
string_to_int={ch:i for i, ch in enumerate(char)}
int_to_string={i:ch for i, ch in enumerate(char)}
encode=lambda s:[string_to_int[c] for c in s]
decode=lambda s:"".join([int_to_string[c] for c in s])

In [128]:
data = torch.tensor(encode(text), dtype=torch.long)

In [129]:
n=int(len(text)*0.8)
train_data=data[:n]
val_data=data[n:]

In [130]:
#Get batch function
def get_batch(split):
    data=train_data if split=='train' else val_data
    ix=torch.randint(len(data)-block_size,(batch_size,))
    #print(ix)
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y


In [131]:
class Head(nn.Module):
    def __init__(self,head_size):
        super().__init__()
        self.key=nn.Linear(n_embd,head_size,bias=False)
        self.query=nn.Linear(n_embd,head_size,bias=False)
        self.value=nn.Linear(n_embd,head_size,bias=False)
        self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))

        self.dropout=nn.Dropout(droput)
    def forward(self,x):
        B,T,C=x.shape
        k=self.key(x)
        q=self.query(x)

        wei=q@k.transpose(-2,1)*k.shape[-1]**-0.5
        wei=wei.masked_fill(self.tril[:T,:T]==0,float('-inf'))
        wei=F.softmax(wei,dim=-1)
        wei.self.dropout(wei)
        v=self.values(x)
        out=wei@v
        return out

In [132]:
class FeedForward(nn.Module):
    def __init__(self,n_embd):
        super().__init__() #This line calls the constructor of the parent class (nn.Module) to initialize the module.
        self.net=nn.Sequential(
            nn.Linear(n_embd,4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd,n_embd),
            nn.Dropout(dropout)
        )
    def forword(self,x):
        return self.net(x)

In [133]:
class MultiheadAttention(nn.Module):
    def __init__(self,num_heads,head_size):
        super().__init__() #This line calls the constructor of the parent class (nn.Module) to initialize the module.
        self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj=nn.Linear(head_size*num_heads,n_embd)
        self.dropout=nn.Dropout(droput)

    def forward(self,x):
        out=torch.cat([h(x) for h in self.heads])  
        out=self.dropout(self.proj(out))
        return out  

In [134]:
class Block(nn.Module):
    '''Transformer BLock'''
    def __init__(self,n_embd,n_head):
        super().__init__() #This line calls the constructor of the parent class (nn.Module) to initialize the module.
        head_size=n_embd//n_head
        self.sa=MultiheadAttention(n_head,head_size)   #self attention
        self.ffwd=FeedForward(n_embd)   
        self.ln1=nn.LayerNorm(n_embd)
        self.ln2=nn.LayerNorm(n_embd)
    
    def forward(self,x):
        y=self.sa(x)             #self attention
        x=self.ln1(x+y)          #Norm 1
        y=self.ffwd(x)           #feed forward
        x=self.ln2(x+y)          #Norm 2
        return x

In [135]:
class GptLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__() #This line calls the constructor of the parent class (nn.Module) to initialize the module.
        self.token_embedding_table=nn.Embedding(vocab_size,n_embd)
        self.position_embedding_table=nn.Embedding(block_size,n_embd)
        self.block=nn.Sequential(*[Block(n_embd,n_head=n_head) for _ in range(n_layer)])
        self.ln_f=nn.LayerNorm(n_embd) #final layer norm
        self.lm_head=nn.Linear(n_embd,vocab_size)

       # self.apply(self.__init__weights)
    
    def __init__weights(self,module):
        if isinstance(module,nn.Linear):
            torch.nn.init.normal_(module.weight,mean=0.0,std=0.0)
        if module.bias is not None:
            torch.nn.init.zeros(module.bias)
        elif isinstance(module,nn.Embedding):
            torch.nn.init.normal_(module.weight,mean=0.0,std=0.0)
    
    def forward(self,index,target=None):
       
       token_emb=self.token_embedding_table(idx)
       pos_emb=self.position_embedding_table(torch.arrange(T)) 
       x=token_emb+pos_emb
       x=self.block(x)
       x=self.ln_f(x)
       logit=self.lm_head(x)

       if target==None:
            loss=None
       else:
         #geting shape of logits
         B, T, C=logit.shape
         logit=logit.view(B*T,C)
         target=target.view(B*T)
         loss=F.cross_entropy(logit,target)
        
       return logit, loss
    
    def generate(self,index,max_new_tokens):
        #index is (B,T) array of indices in the current content
        for _ in range(max_new_tokens):
            #get prediction
            logit, loss=self.forward(index)
            #focus only on last step
            logit=logit[:,-1,:] #becomes (B,C)
            probs=F.softmax(logit,dim=-1) #(B,C)
            index_next=torch.multinomial(probs,num_samples=1)
            #append sampled index to the running sequence
            index=torch.cat((index,index_next),dim=1)
        return index


In [136]:
model=GptLanguageModel(vocab_size)

In [137]:
print(torch.empty(1,1,1))

tensor([[[0.]]])


In [138]:
context=torch.zeros((1,1),dtype=torch.long,)

In [139]:

optimizer=torch.optim.AdamW(model.parameters(),lr=learning_rate)
losst=[]

for iter in range(max_iter*2):
    xb, yb=get_batch('train')

    #Evluate loss

    logit, loss=model.forward(xb,yb)
    losst.append(loss)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss)

NameError: name 'idx' is not defined